In [8]:
import pandas as pd
from Bio.Seq import Seq

# Define the barcode components
flow_cell_p5 = Seq("AATGATACGGCGACCACCGAGATCTA")
flow_cell_p7 = Seq("CAAGCAGAAGACGGCATACGAGAT")
seq_primer_r1 = Seq("CACTCTTTCCCTACACGACGCTCTTCCGATCT")
seq_primer_r2 = Seq("GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT")

In [9]:
def main(f_primer_st,r_primer_st,i7_index_st):
    # Define the custom forward and reverse primers
    #print(f_primer_st)
    f_primer = Seq(f_primer_st) # Example custom forward primer
    r_primer = Seq(r_primer_st) # Example custom reverse primer

    # Define the I7 index barcode
    i7_index = Seq(i7_index_st) # Example I7 index barcode

    # Generate the sequencing primers
    p5 = flow_cell_p5 + seq_primer_r1
    p7 = flow_cell_p7 + i7_index.reverse_complement()+seq_primer_r2

    # Generate the complete forward and reverse primers
    f_primer_with_barcode = p5 + f_primer
    r_primer_with_barcode = p7 + r_primer

    # Print the resulting primers
    return str(f_primer_with_barcode), str(r_primer_with_barcode)


In [10]:
def convert_to_plate_position(number):
    if number < 1 or number > 96:
        return "Invalid input. Number should be between 1 and 96."
    
    row = chr((number - 1) // 12 + 65)  # Convert to corresponding ASCII character
    column = (number - 1) % 12 + 1
    
    return f"{row}{column}"

In [11]:
def get_primer(df,out_name):
    name = []
    whole = []
    pos = []
    for x in range(len(df)):
        f,r = main(df['For_match'][x].lower(),df['Rev_match'][x].lower(),df['Code'][x].lower())
        name.append(df['Site'][x]+'_F')
        whole.append(f)
        pos.append(convert_to_plate_position(x+1))
        name.append(df['Site'][x]+'_R')
        whole.append(r)
        pos.append(convert_to_plate_position(x+1))
    dict_ = {'pos':pos,'name':name,'seq':whole}
    df2 = pd.DataFrame.from_dict(dict_)
    df2.to_csv(out_name,index=False)

In [14]:
input_csv_path = "batch7_test.csv"
out_name = 'primer_novelseq_batch7_idt.csv'

In [15]:
df= pd.read_csv('batch7_test.csv',sep=',')
get_primer(df,out_name)